## Reference
* 본 자료는 lovit님의 fasttext tutorial 자료를 따라하며 공부한 것임을 밝혀둡니다.

## Build Up
* windows에서 fasttext build up은 https://medium.com/@juneoh/windows%EC%9A%A9-fasttext-%EB%B0%94%EC%9D%B4%EB%84%88%EB%A6%AC%EB%93%A4-727829b010a를 참조하였습니다.

fasttext를 학습하기 위해서는 두 가지 방법이 있습니다.<br>
* gensim에서 제공하는 FastText 이용하기
* Facebook에서 제공하는 모듈 사용하기<br>

이번 tutorial에서는 gensim에서 제공하는 FastText를 이용하여 학습을 해봅니다.

한글에 fasttext를 적용하려면 초/중/종성을 분리해야 한다. 따라서 이를 수행하는 함수를 먼저 정의해보자.

In [3]:
# import config
from soynlp.hangle import decompose, compose
from soynlp.normalizer import remove_doublespace

## lovit님의 function

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

In [4]:
print(encode('자연어처리는 어렵기도 하고 재미있어요'))
print(decode(encode('자연어처리는 어렵기도 하고 재미있어요')))

ㅈㅏ-ㅇㅕㄴㅇㅓ-ㅊㅓ-ㄹㅣ-ㄴㅡㄴ ㅇㅓ-ㄹㅕㅂㄱㅣ-ㄷㅗ- ㅎㅏ-ㄱㅗ- ㅈㅐ-ㅁㅣ-ㅇㅣㅆㅇㅓ-ㅇㅛ-
자연어처리는 어렵기도 하고 재미있어요


fasttext의 input은 word2vec과 유사한 형태로, list안에 각 문서(리뷰)별로 list가 있으며, 리뷰들이 tokenize된 형태이다.<br>
하지만 word2vec과 다른점은 초/중/종성으로 tokenize된 형태라는 것이다.<br>
따라서 네이버 영화 리뷰 데이터를 아래와 같이 초중종성을 분리해서 fasttext input으로 만들어보자.

In [5]:
path = 'C:/Users/sbh0613/Desktop/NLP/dataset/ratings_train.txt'
with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
naver_comments = [i.split('\t')[1] for i in lines][1:]

fasttext_corpus = [ encode(i).split(' ') for i in naver_comments]

In [6]:
import pickle
with open('C:/Users/sbh0613/Desktop/NLP/dataset/ratings_train_preprocessed_fasttext.txt','wb') as f:
    pickle.dump(fasttext_corpus, f)

In [7]:
from gensim.models import FastText

fasttext_model = FastText(
        fasttext_corpus,
        window = 3,
        min_count = 10,
        min_n = 3,
        max_n = 6
    )

gensim의 fasttext의 model은 word2vec의 model 사용법과 유사하다. 즉, word vector을 뽑으려면 .wv를 해야한다는 점, 그리고 가장 비슷한 단어(fasttext에서는 초/중/종성이 분리된 단어)도 .most_similar을 통해 얻을 수 있다.

fasttext의 input으로 추/중/종성으로 분해된 단어를 입력했으니, fasttext model에 학습된 단어도 초/중/종성이 분해된 형태이다. 그 형태는 아래와 같다.

In [20]:
list(fasttext_model.wv.vocab.keys())[:10]

['ㅇㅏ-',
 'ㄷㅓ-ㅂㅣㅇ',
 'ㅈㅣㄴㅉㅏ-',
 'ㅉㅏ-ㅈㅡㅇㄴㅏ-ㄴㅔ-ㅇㅛ-',
 'ㅁㅗㄱㅅㅗ-ㄹㅣ-',
 'ㅎㅡㅁ',
 'ㅍㅗ-ㅅㅡ-ㅌㅓ-ㅂㅗ-ㄱㅗ-',
 'ㄱㅏ-ㅂㅕㅂㅈㅣ-',
 'ㅅㅗㄹㅈㅣㄱㅎㅣ-',
 'ㅈㅐ-ㅁㅣ-ㄴㅡㄴ']

초/중/종성이 분리된 형태가 아니라 우리가 익숙한 단어의 형태로 바꿔주는 형태로 복원해보자.

In [21]:
def most_similar_agg(fasttext_model, hangle, topn = 10):
    hangle = encode(hangle)
    most_sim = fasttext_model.wv.most_similar(hangle, topn = topn)
    most_sim_agg = [(decode(word), sim)  for word, sim in most_sim]
    return most_sim_agg

In [22]:
most_similar_agg(fasttext_model, '영홯', topn=10)

[('영활', 0.9968951940536499),
 ('영환', 0.9958025217056274),
 ('영화계', 0.9909502863883972),
 ('영화평', 0.9877427220344543),
 ('영화속', 0.9873830080032349),
 ('영화판', 0.9871527552604675),
 ('영화군', 0.9871389269828796),
 ('영화란', 0.9855873584747314),
 ('영화화', 0.9830932021141052),
 ('영화랑', 0.9801906943321228)]

In [23]:
most_similar_agg(fasttext_model, '쓰뤠', topn=10)

[('쓰렉', 0.9919682741165161),
 ('쓰레기네', 0.9732194542884827),
 ('쓰레기야', 0.9635607004165649),
 ('쓰레기라', 0.9611065983772278),
 ('쓰레기', 0.9581952095031738),
 ('쓰레기임', 0.9467074871063232),
 ('쓰레기로', 0.9234054684638977),
 ('쓰레기인', 0.9211544990539551),
 ('쓰레기들', 0.9206268191337585),
 ('쓰레기통에', 0.8906248211860657)]